# Proyecto Modelos y Simulación de Sistemas I

In [1]:
import polars as pl


In [2]:
csv_file = "udea-ai-4-eng-20251-pruebas-saber-pro-colombia/train.csv"

### Cargar el csv

In [3]:
train_df = pl.scan_csv(csv_file)

In [4]:
first_two_rows = train_df.head(2).collect()

display(first_two_rows)

ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,coef_1,coef_2,coef_3,coef_4
i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,f64
904256,20212,"""ENFERMERIA""","""BOGOTÁ""","""Entre 5.5 millones y menos de …","""Menos de 10 horas""","""Estrato 3""","""Si""","""Técnica o tecnológica incomple…","""Si""","""Si""","""N""","""No""","""Si""","""Si""","""Postgrado""","""medio-alto""",0.322,0.208,0.31,0.267
645256,20212,"""DERECHO""","""ATLANTICO""","""Entre 2.5 millones y menos de …","""0""","""Estrato 3""","""No""","""Técnica o tecnológica completa""","""Si""","""No""","""N""","""No""","""Si""","""No""","""Técnica o tecnológica incomple…","""bajo""",0.311,0.215,0.292,0.264


| Columna                      | Tipo de Variable      | Acción de Codificación                                                                                                                                   |
|------------------------------|-----------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------|
| `RENDIMIENTO_GLOBAL`         | Ordinal (Target)      | Mapear a valores numéricos `0`–`3` según el orden (e.g., `"bajo": 0`, `"medio-bajo": 1`, `"medio-alto": 2`, `"alto": 3`)                                  |
| `FAMI_ESTRATOVIVIENDA`       | Ordinal               | Mapear `"Estrato 1"`…`"Estrato 6"` a valores numéricos `1`…`6`                                                                                           |
| `ESTU_HORASSEMANATRABAJA`    | Ordinal               | Mapear rangos de texto (e.g., `"0"`, `"Menos de 10 horas"`, etc.) a valores numéricos ordinales                                                          |
| `FAMI_TIENEINTERNET`         | Categórica Binaria    | Convertir a `0` (No) / `1` (Si) y castear a `UInt8`                                                                                                      |
| `FAMI_TIENELAVADORA`         | Categórica Binaria    | Convertir a `0` (No) / `1` (Si) y castear a `UInt8`                                                                                                      |
| `FAMI_TIENEAUTOMOVIL`        | Categórica Binaria    | Convertir a `0` (No) / `1` (Si) y castear a `UInt8`                                                                                                      |
| `ESTU_PRIVADO_LIBERTAD`      | Categórica Binaria    | Convertir a `0` (N) / `1` (S) y castear a `UInt8`                                                                                                        |
| `ESTU_PAGOMATRICULAPROPIO`   | Categórica Binaria    | Convertir a `0` (No) / `1` (Si) y castear a `UInt8`                                                                                                      |
| `FAMI_TIENECOMPUTADOR`       | Categórica Binaria    | Convertir a `0` (No) / `1` (Si) y castear a `UInt8`                                                                                                      |
| `FAMI_TIENEINTERNET.1`       | Categórica Binaria    | Convertir a `0` (No) / `1` (Si) y castear a `UInt8` —  **Revisar si es columna duplicada**                                                                  |
| `FAMI_EDUCACIONPADRE`        | Categórica Nominal    | Aplicar One-hot Encoding o Target Encoding                                                                                                               |
| `FAMI_EDUCACIONMADRE`        | Categórica Nominal    | Aplicar One-hot Encoding o Target Encoding                                                                                                               |
| `ESTU_PRGM_ACADEMICO`        | Categórica Nominal    | Aplicar One-hot Encoding o Target Encoding                                                                                                               |
| `ESTU_PRGM_DEPARTAMENTO`     | Categórica Nominal    | Aplicar One-hot Encoding o Target Encoding                                                                                                               |
| `coef_1` … `coef_4`          | Continua              | Mantener como `Float64`                                                                                                                                 |
| Otras columnas (`ID`, `PERIODO`, etc.) | Variables de Identificación | No hacer nada por ahora                                                                                         |


## Columnas Repetidas

Parece que la columna **FAMI_TIENEINTERNET** está repetida

In [5]:
equal_query = (
    train_df
    .select(
        #Seleccionar las columnas que se van a comparar
        (pl.col("FAMI_TIENEINTERNET") == pl.col("FAMI_TIENEINTERNET.1"))
        .all()
        .alias("equal_query")
    )
    .collect()
)

equal_query_result = equal_query["equal_query"][0]

print(f"Son todos los varlores 'FAMI_TIENEINTERNET' iguales a 'FAMI_TIENEINTERNET.1'? {equal_query_result}")

Son todos los varlores 'FAMI_TIENEINTERNET' iguales a 'FAMI_TIENEINTERNET.1'? True


Vemos que, como efectivamente **está repetida**, la podemos borrar.

In [6]:
train_df = train_df.drop("FAMI_TIENEINTERNET.1")

## Estandarización

En esta sección, vamos a analizar problemas que pensamos que pueden tener algunas columnas particulares como lo son las ciudades y los programas académicos

### Ciudades

In [7]:
departments_count = (
    train_df
    .select(
        pl.col("ESTU_PRGM_DEPARTAMENTO")
          .n_unique()
          .alias("n_departamentos")
    )
    .collect()
)["n_departamentos"][0]
print(f"Hay {departments_count} departamentos distintos.")

Hay 31 departamentos distintos.


In [8]:
departaments = (
    train_df
    .select(
        pl.col("ESTU_PRGM_DEPARTAMENTO")
          .unique()
          .sort()
          .alias("departamentos")
    )
    .collect()
)["departamentos"]

# Convertimos la serie de departamentos a una lista
lista_departamentos = departaments.to_list()
print(f"{len(lista_departamentos)} departamentos encontrados:")
# Un salto de línea para mejor legibilidad
print("\n".join(lista_departamentos))

31 departamentos encontrados:
AMAZONAS
ANTIOQUIA
ARAUCA
ATLANTICO
BOGOTÁ
BOLIVAR
BOYACA
CALDAS
CAQUETA
CASANARE
CAUCA
CESAR
CHOCO
CORDOBA
CUNDINAMARCA
GUAVIARE
HUILA
LA GUAJIRA
MAGDALENA
META
NARIÑO
NORTE SANTANDER
PUTUMAYO
QUINDIO
RISARALDA
SAN ANDRES
SANTANDER
SUCRE
TOLIMA
VALLE
VAUPES


No evidenciamos problemas importantes con los departamentos

### Programas

In [9]:
unique_programs = (
    train_df
    .select(
        pl.col("ESTU_PRGM_ACADEMICO")
          .str.replace(r'^"|"$', '', literal=False)           # quita comillas al inicio/final
          .str.replace(r'^\s+|\s+$', '', literal=False)  # quita espacios al inicio/final
          .str.to_uppercase()                                 
    )
    .unique()
    .sort("ESTU_PRGM_ACADEMICO")
    .collect()
    .get_column("ESTU_PRGM_ACADEMICO")
    .to_list()
)


#### Distancia de Levenshtein para Programas académicos

In [10]:
def levenshtein(a: str, b: str) -> int:
    if len(a) < len(b):
        a, b = b, a
    if not b:
        return len(a)
    prev = list(range(len(b) + 1))
    for i, ca in enumerate(a, start=1):
        curr = [i]
        for j, cb in enumerate(b, start=1):
            ins = prev[j] + 1
            rem = curr[j-1] + 1
            rep = prev[j-1] + (ca != cb)
            curr.append(min(ins, rem, rep))
        prev = curr
    return prev[-1]

In [11]:
threshold = 2
suggestions = []
n = len(unique_programs)
for i in range(n):
    for j in range(i+1, n):
        p1, p2 = unique_programs[i], unique_programs[j]
        d = levenshtein(p1.lower(), p2.lower())
        if d <= threshold:
            suggestions.append((p1, p2, d))


In [12]:
# Imprimimos las sugerencias de corrección
print(f"\nSugerencias de corrección (distancia <= {threshold}):")
for p1, p2, d in suggestions:
    print(f"{p1} <-> {p2} (distancia: {d})")


Sugerencias de corrección (distancia <= 2):
ADMINISTRACION  FINANCIERA <-> ADMINISTRACION FINANCIERA (distancia: 1)
ADMINISTRACION  FINANCIERA <-> ADMINISTRACIÓN FINANCIERA (distancia: 2)
ADMINISTRACION COMERCIAL <-> ADMINISTRACIÓN COMERCIAL (distancia: 1)
ADMINISTRACION DE COMERCIO EXTERIOR <-> ADMINISTRACIÓN DE COMERCIO EXTERIOR (distancia: 1)
ADMINISTRACION DE EMPRESAS <-> ADMINISTRACI¿N DE EMPRESAS (distancia: 1)
ADMINISTRACION DE EMPRESAS <-> ADMINISTRACIÓN DE EMPRESAS (distancia: 1)
ADMINISTRACION DE EMPRESAS AGROINDUSTRIALES <-> ADMINISTRACIÓN DE EMPRESAS AGROINDUSTRIALES (distancia: 1)
ADMINISTRACION DE EMPRESAS AGROPECUARIAS <-> ADMINISTRACIÓN DE EMPRESAS AGROPECUARIAS (distancia: 1)
ADMINISTRACION DE EMPRESAS TURISTICA <-> ADMINISTRACION DE EMPRESAS TURISTICAS (distancia: 1)
ADMINISTRACION DE EMPRESAS TURISTICA <-> ADMINISTRACIÓN DE EMPRESAS TURISTICAS (distancia: 2)
ADMINISTRACION DE EMPRESAS TURISTICAS <-> ADMINISTRACIÓN DE EMPRESAS TURISTICAS (distancia: 1)
ADMINISTRACION

Evidenciamos que los principales problemas son con tildes y espacios intermedios. Vamos a eliminarlos

In [13]:
accent_map = {
    "á": "a", "é": "e", "í": "i", "ó": "o", "ú": "u",
    "Á": "A", "É": "E", "Í": "I", "Ó": "O", "Ú": "U",
    "ñ": "n", "Ñ": "N"
}

# Construimos la expresión de normalización
expr = (
    pl.col("ESTU_PRGM_ACADEMICO")
      # 1) quitar comillas
      .str.replace_all('"', '', literal=True)
      .str.to_uppercase()
)

# encadenar remap de cada tilde
for orig, repl in accent_map.items():
    expr = expr.str.replace_all(orig, repl, literal=True)

# eliminar cualquier carácter que no sea A–Z, dígitos o espacio
expr = expr.str.replace_all(r'[^A-Z0-9 ]', '', literal=False)

# colapsar múltiples espacios
expr = expr.str.replace_all(r'\s{2,}', ' ', literal=False)

# uitar espacios al inicio/fin
expr = expr.str.replace_all(r'^\s+|\s+$', '', literal=False)

# renombramos
expr = expr.alias("prog_norm")

# Ejecutamos lazy y obtenemos la lista limpia:
normalized_programs = (
    train_df
    .select(expr)
    .unique()
    .sort("prog_norm")
    .collect()
    .get_column("prog_norm")
    .to_list()
)

print(f"{len(normalized_programs)} programas normalizados:")
print(normalized_programs[:20], "…")

752 programas normalizados:
['3 CICLO PROFESIONAL NEGOCIOS INTERNACIONALES', 'ACTIVIDAD FISICA Y DEPORTE', 'ACUICULTURA', 'ADMINISTRACIN DE EMPRESAS', 'ADMINISTRACIN DE NEGOCIOS INTERNACIONALES', 'ADMINISTRACIN LOGSTICA', 'ADMINISTRACIN PBLICA', 'ADMINISTRACION', 'ADMINISTRACION AERONAUTICA', 'ADMINISTRACION AGROINDUSTRIAL', 'ADMINISTRACION AGROPECUARIA', 'ADMINISTRACION AMBIENTAL', 'ADMINISTRACION AMBIENTAL Y DE LOS RECURSOS NATURALES', 'ADMINISTRACION BANCARIA Y FINANCIERA', 'ADMINISTRACION COMERCIAL', 'ADMINISTRACION COMERCIAL Y DE MERCADEO', 'ADMINISTRACION COMERCIAL Y FINANCIERA', 'ADMINISTRACION DE AGRONEGOCIOS', 'ADMINISTRACION DE COMERCIO EXTERIOR', 'ADMINISTRACION DE EMPRESAS'] …


In [14]:
threshold = 3
suggestions = []
n = len(normalized_programs)
for i in range(n):
    for j in range(i+1, n):
        p1, p2 = normalized_programs[i], normalized_programs[j]
        d = levenshtein(p1.lower(), p2.lower())
        if d <= threshold:
            suggestions.append((p1, p2, d))


In [15]:
print( f"\nSugerencias de corrección (normalizados, distancia <= {threshold}):")
for p1, p2, d in suggestions:
    print(f"{p1} <-> {p2} (distancia: {d})")


Sugerencias de corrección (normalizados, distancia <= 3):
ADMINISTRACIN DE EMPRESAS <-> ADMINISTRACION DE EMPRESAS (distancia: 1)
ADMINISTRACIN DE EMPRESAS <-> ADMINSITRACION DE EMPRESAS (distancia: 3)
ADMINISTRACIN DE NEGOCIOS INTERNACIONALES <-> ADMINISTRACION DE NEGOCIOS INTERNACIONALES (distancia: 1)
ADMINISTRACIN DE NEGOCIOS INTERNACIONALES <-> ADMINISTRACION EN NEGOCIOS INTERNACIONALES (distancia: 3)
ADMINISTRACIN DE NEGOCIOS INTERNACIONALES <-> ADMINISTRACION Y NEGOCIOS INTERNACIONALES (distancia: 3)
ADMINISTRACIN LOGSTICA <-> ADMINISTRACION LOGISTICA (distancia: 2)
ADMINISTRACIN PBLICA <-> ADMINISTRACION PUBLICA (distancia: 2)
ADMINISTRACION DE EMPRESAS <-> ADMINSITRACION DE EMPRESAS (distancia: 2)
ADMINISTRACION DE EMPRESAS AGROINDUSTRIALES <-> ADMINISTRACION EMPRESAS AGROINDUSTRIALES (distancia: 3)
ADMINISTRACION DE EMPRESAS TURISTICA <-> ADMINISTRACION DE EMPRESAS TURISTICAS (distancia: 1)
ADMINISTRACION DE MERCADEO Y LOGISTICA INTERNACIONALES <-> ADMINISTRACION EN MERCADEO

Vamos a mapear manualmente los reemplazos

In [16]:
mapping = {
    # ADMINISTRACIÓN
    "ADMINISTRACIN DE EMPRESAS":              "ADMINISTRACION DE EMPRESAS",
    "ADMINISTRACIN DE NEGOCIOS INTERNACIONALES": "ADMINISTRACION DE NEGOCIOS INTERNACIONALES",
    "ADMINISTRACIN LOGSTICA":                 "ADMINISTRACION LOGISTICA",
    "ADMINISTRACIN PBLICA":                   "ADMINISTRACION PUBLICA",
    "ADMINSITRACION DE EMPRESAS":             "ADMINISTRACION DE EMPRESAS",
    "ADMINISTRACION DE EMPRESAS TURISTICA":   "ADMINISTRACION DE EMPRESAS TURISTICAS",
    "ADMINISTRACION DE MERCADEO Y LOGISTICA INTERNACIONALES":
        "ADMINISTRACION EN MERCADEO Y LOGISTICA INTERNACIONALES",
    # VARIANTES “EN” vs “DE”
    "ADMINISTRACION DE NEGOCIOS INTERNACIONALES": "ADMINISTRACION EN NEGOCIOS INTERNACIONALES",
    "ADMINISTRACION DE SERVICIOS DE SALUD":       "ADMINISTRACION EN SERVICIOS DE SALUD",
    "ADMINISTRACION TURISTICA Y HOTELERA":        "ADMINISTRACION TURISTICA Y HOTELERA",  # ya estaba ok
    # AGRONOMÍA / ASTRONOMÍA / GASTRONOMÍA
    "AGRONOMIA":      "AGRONOMIA",
    "ASTRONOMIA":     "ASTRONOMIA",
    # BIOLOGÍA / ECOLOGÍA / GEOLOGÍA / TEOLOGÍA
    "BIOLOGIA":       "BIOLOGIA",
    "ECOLOGIA":       "ECOLOGIA",
    "GEOLOGIA":       "GEOLOGIA",
    "TEOLOGIA":       "TEOLOGIA",
    # CIENCIAS POLÍTICAS
    "CIENCIA POLITICA":              "CIENCIAS POLITICAS",
    # COMUNICACIÓN
    "COMUNICACIN SOCIAL":            "COMUNICACION SOCIAL",
    "COMUNICACIN SOCIAL Y PERIODISMO": "COMUNICACION SOCIAL Y PERIODISMO",
    "COMUNICACIN SOCIAL PERIODISMO": "COMUNICACION SOCIAL Y PERIODISMO",
    "COMUNICACION SOCIALY PERIODISMO":"COMUNICACION SOCIAL Y PERIODISMO",
    "COMUNICACIN VISUAL":            "COMUNICACION VISUAL",
    "COMUNICACION":                   "COMUNICACIONES",
    "COMUNICACION AUDIOVISUAL Y MULTIMEDIAL":
        "COMUNICACION AUDIOVISUAL Y MULTIMEDIOS",
    # CONTADURÍA PÚBLICA
    "CONTADURIA PBLICA":             "CONTADURIA PUBLICA",
    # DEPORTE
    "DEPORTE Y ACTIVIDAD FISICA":     "DEPORTE Y ACTIVIDAD FISICA",
    "DEPORTE Y ACTIVIDADA FISICA":    "DEPORTE Y ACTIVIDAD FISICA",
    # DISEÑO
    "DISENO CROSSMEDIA":              "DISENO CROSSMEDIA",
    "DISEO CROSSMEDIA":               "DISENO CROSSMEDIA",
    "DISENO DE MODA":                 "DISENO DE MODAS",
    "DISENO GRAFICO":                 "DISENO GRAFICO",
    "DISEÑO GRAFICO":                 "DISENO GRAFICO",
    # ECONOMÍA / ECOLOGÍA / GEOLOGÍA / TEOLOGÍA
    "ECONOMA":                        "ECONOMIA",
    # INGENIERÍA
    "INGENIERA DE SISTEMAS":          "INGENIERIA DE SISTEMAS",
    "INGENIERA ELCTRICA":             "INGENIERIA ELECTRICA",
    "INGENIERA EN SOFTWARE":          "INGENIERIA EN SOFTWARE",
    "INGENIERA INDUSTRIAL":           "INGENIERIA INDUSTRIAL",
    "INGENIERA INFORMTICA":           "INGENIERIA INFORMATICA",
    "INGENIERIA BIOLOGICA":           "INGENIERIA BIOLOGICA",
    "INGENIERIA GEOLOGICA":           "INGENIERIA GEOLOGICA",
    "INGENIERIA DE CONTROL":          "INGENIERIA EN CONTROL",
    "INGENIERIA DE PROCESOS INDUSTRIALES":
        "INGENIERIA EN PROCESOS INDUSTRIALES",
    "INGENIERIA DE SOFTWARE":         "INGENIERIA EN SOFTWARE",
    "INGENIIERIA DE SOFTWARE":        "INGENIERIA DE SOFTWARE",
    "INGENIERIA DE TELECOMUNICACIONES":
        "INGENIERIA EN TELECOMUNICACIONES",
    "INGENIERIA ELECTRICA":           "INGENIERIA ELECTRICA",
    "INGENIERIA ELECTRONICA":         "INGENIERIA ELECTRONICA",
    "INGENIERIA EN ENERGIA":          "INGENIERIA EN ENERGIAS",
    "INGENIERIA MECATRONICA":         "INGENIERIA MECATRONICA",
    "INGENIERIA MECATRONICO":         "INGENIERIA MECATRONICA",
    # INSTRUMENTACIÓN
    "INSTRUMENTACION QUIRURGICA":     "INSTRUMENTACION QUIRURGICA",
    "INTRUMENTACION QUIRURGICA":      "INSTRUMENTACION QUIRURGICA",
    # LICENCIATURAS
    "LICENCIATURA EN ARTES ESCENICAS": 
        "LICENCIATURA EN ARTES ESCENICAS",
    "LICENCIATURA EN ARTES ESCNICAS": 
        "LICENCIATURA EN ARTES ESCENICAS",
    "LICENCIATURA EN BIOLOGIA":       "LICENCIATURA EN BIOLOGIA",
    "LICENCIATURA EN EDUCACIN ARTSTICA":
        "LICENCIATURA EN EDUCACION ARTISTICA",
    "LICENCIATURA EN EDUCACIN BSICA PRIMARIA":
        "LICENCIATURA EN EDUCACION BASICA PRIMARIA",
    "LICENCIATURA EN EDUCACIN INFANTIL":
        "LICENCIATURA EN EDUCACION INFANTIL",
    "LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN EDUCACION FISICA RECREACION Y DEPORTES":
        "LICENCIATURA EN EDUCACION BASICA CON ENFASIS EN EDUCACION FISICA RECREACION Y DEPORTES",
    "LICENCIATURA EN EDUCACION FISICA RECREACION Y DEPORTE":
        "LICENCIATURA EN EDUCACION FISICA RECREACION Y DEPORTES",
    "LICENCIATURA EN EDUCACION FISICARECREACION Y DEPORTE":
        "LICENCIATURA EN EDUCACION FISICA RECREACION Y DEPORTES",
    "LICENCIATURA EN EDUCACON FISICA RECREACION Y DEPORTES":
        "LICENCIATURA EN EDUCACION FISICA RECREACION Y DEPORTES",
    "LICENCIATURA EN FILOSOFA Y HUMANIDADES":
        "LICENCIATURA EN FILOSOFIA Y HUMANIDADES",
    "LICENCIATURA EN INGLES ESPANOL":
        "LICENCIATURA EN INGLES ESPANOL",
    "LICENCIATURA EN LENGUAS EXTRANJERAS CON NFASIS EN INGLS":
        "LICENCIATURA EN LENGUAS EXTRANJERAS CON ENFASIS EN INGLES",
    "LICENCIATURA EN LENGUAS EXTRANJERAS INGLESFRANCES":
        "LICENCIATURA EN LENGUAS EXTRANJERAS INGLES FRANCES",
    "LICENCIATURA EN MATEMATICA APLICADA":
        "LICENCIATURA EN MATEMATICAS APLICADAS",
    "LICENCIATURA EN MATEMTICAS":
        "LICENCIATURA EN MATEMATICAS APLICADAS",
    "LICENCIATURA EN PEDAGOGA INFANTIL":
        "LICENCIATURA EN PEDAGOGIA INFANTIL",
    # OTROS
    "CIENCIA DE LA INFORMACION BIBLIOTECOLOGIA":
        "CIENCIA DE LA INFORMACION Y BIBLIOTECOLOGIA",
    "CIENCIA POLITICA":                "CIENCIAS POLITICAS",
    "COMUNICACION SOCIALPERIODISMO":   "COMUNICACION SOCIAL Y PERIODISMO",
    "COMUNICACION SOCIAL PERIODISMO":  "COMUNICACION SOCIAL Y PERIODISMO",
    "COMUNICACION SOCIAL Y PERIODISMO": "COMUNICACION SOCIAL Y PERIODISMO",
    "COMUNICACION":                    "COMUNICACIONES",
    "COMUNICACION AUDIOVISUAL Y MULTIMEDIAL":
        "COMUNICACION AUDIOVISUAL Y MULTIMEDIOS",
    "DISENO DE MODA":                  "DISENO DE MODAS",
    "DISEO DE MODA":                   "DISENO DE MODAS",
    "DISEÑO GRAFICO":                  "DISENO GRAFICO",
    "ECOLOGIA":                        "ECOLOGIA",
    "GEOLOGA":                         "GEOLOGIA",
    "LICENCIATURA EN FISICA":          "LICENCIATURA EN FISICA",
    "LICENCIATURA EN MUSICA":          "LICENCIATURA EN MUSICA",
    "PROFESIONAL EN GASTRONOMA":       "PROFESIONAL EN GASTRONOMIA",
    "PSICOLOGA":                       "PSICOLOGIA",
    "QUIMICA FARMACEUTICA":            "QUIMICA FARMACEUTICA",
    "QUMICA FARMACUTICA":              "QUIMICA FARMACEUTICA",
}


In [17]:
prog_expr = (
    pl.col("ESTU_PRGM_ACADEMICO")
      .str.replace_all('"',     '', literal=True)
      .str.to_uppercase()
)

for o, r in accent_map.items():
    prog_expr = prog_expr.str.replace_all(o, r, literal=True)

prog_expr = (
    prog_expr
      .str.replace_all(r'[^A-Z0-9 ]', '', literal=False)
      .str.replace_all(r'\s{2,}',      ' ',  literal=False)
      .str.replace_all(r'^\s+|\s+$',   '',  literal=False)
      .replace(mapping)
      .alias("prog_norm")
)

train_df = train_df.with_columns([prog_expr])

print(train_df.collect_schema())

preview = (
    train_df
    .select(["ESTU_PRGM_ACADEMICO", "prog_norm"])
    .limit(10)
    .collect()
)
print(preview)

Schema([('ID', Int64), ('PERIODO', Int64), ('ESTU_PRGM_ACADEMICO', String), ('ESTU_PRGM_DEPARTAMENTO', String), ('ESTU_VALORMATRICULAUNIVERSIDAD', String), ('ESTU_HORASSEMANATRABAJA', String), ('FAMI_ESTRATOVIVIENDA', String), ('FAMI_TIENEINTERNET', String), ('FAMI_EDUCACIONPADRE', String), ('FAMI_TIENELAVADORA', String), ('FAMI_TIENEAUTOMOVIL', String), ('ESTU_PRIVADO_LIBERTAD', String), ('ESTU_PAGOMATRICULAPROPIO', String), ('FAMI_TIENECOMPUTADOR', String), ('FAMI_EDUCACIONMADRE', String), ('RENDIMIENTO_GLOBAL', String), ('coef_1', Float64), ('coef_2', Float64), ('coef_3', Float64), ('coef_4', Float64), ('prog_norm', String)])
shape: (10, 2)
┌─────────────────────────────────┬─────────────────────────────────┐
│ ESTU_PRGM_ACADEMICO             ┆ prog_norm                       │
│ ---                             ┆ ---                             │
│ str                             ┆ str                             │
╞═════════════════════════════════╪═════════════════════════════════

In [18]:
# Seleccionamos las columnas STU_PRGM_ACADEMICO y prog_norm
display(
    train_df
    .select(["ESTU_PRGM_ACADEMICO", "prog_norm"])
    .limit(100)
    .collect()
)


ESTU_PRGM_ACADEMICO,prog_norm
str,str
"""ENFERMERIA""","""ENFERMERIA"""
"""DERECHO""","""DERECHO"""
"""MERCADEO Y PUBLICIDAD""","""MERCADEO Y PUBLICIDAD"""
"""ADMINISTRACION DE EMPRESAS""","""ADMINISTRACION DE EMPRESAS"""
"""PSICOLOGIA""","""PSICOLOGIA"""
…,…
"""LICENCIATURA EN EDUCACIÓN FÍSI…","""LICENCIATURA EN EDUCACION FISI…"
"""LICENCIATURA EN INGLÉS""","""LICENCIATURA EN INGLES"""
"""DERECHO""","""DERECHO"""


In [19]:
# conteo de programas normalizados
normalized_programs_count = (
    train_df
    .select(pl.col("prog_norm").n_unique().alias("n_programas_normalizados"))
    .collect()
)["n_programas_normalizados"][0]

In [20]:
print(f"Hay {normalized_programs_count} programas normalizados distintos.")

Hay 702 programas normalizados distintos.


## Normalización de columnas Tipo Texto

In [21]:
# lista de columnas de texto a normalizar
text_cols = [
    "ESTU_PRGM_ACADEMICO",
    "ESTU_PRGM_DEPARTAMENTO",
    "ESTU_VALORMATRICULAUNIVERSIDAD",
    "ESTU_HORASSEMANATRABAJA",
    "FAMI_ESTRATOVIVIENDA",
    "FAMI_TIENEINTERNET",
    "FAMI_EDUCACIONPADRE",
    "FAMI_TIENELAVADORA",
    "FAMI_TIENEAUTOMOVIL",
    "ESTU_PRIVADO_LIBERTAD",
    "ESTU_PAGOMATRICULAPROPIO",
    "FAMI_TIENECOMPUTADOR",
    "FAMI_EDUCACIONMADRE",
    "prog_norm"
]

accent_map = {
    r"[áÁ]": "a",
    r"[éÉ]": "e",
    r"[íÍ]": "i",
    r"[óÓ]": "o",
    r"[úÚ]": "u",
    r"[üÜ]": "u",
    r"[ñÑ]": "n",
}

# Construimos un helper para cada columna
def normalize_expr(col_name: str) -> pl.Expr:
    expr = pl.col(col_name)
    # 1) trim espacios
    expr = expr.str.replace(r'^\s+|\s+$', '', literal=False)
    # 2) lowercase
    expr = expr.str.to_lowercase()
    # 3) quitar acentos/ñ
    for pat, rep in accent_map.items():
        expr = expr.str.replace(pat, rep, literal=False)
    return expr.alias(col_name)

# Aplicamos en modo lazy
normalized_df = train_df.with_columns([
    normalize_expr(c) for c in text_cols
])

# mostramos el esquema del DataFrame normalizado
print(normalized_df.collect_schema())
# mostramos un preview de las primeras filas del DataFrame normalizado
display(
    normalized_df
    .select(text_cols)
    .limit(10)
    .collect()
)




Schema([('ID', Int64), ('PERIODO', Int64), ('ESTU_PRGM_ACADEMICO', String), ('ESTU_PRGM_DEPARTAMENTO', String), ('ESTU_VALORMATRICULAUNIVERSIDAD', String), ('ESTU_HORASSEMANATRABAJA', String), ('FAMI_ESTRATOVIVIENDA', String), ('FAMI_TIENEINTERNET', String), ('FAMI_EDUCACIONPADRE', String), ('FAMI_TIENELAVADORA', String), ('FAMI_TIENEAUTOMOVIL', String), ('ESTU_PRIVADO_LIBERTAD', String), ('ESTU_PAGOMATRICULAPROPIO', String), ('FAMI_TIENECOMPUTADOR', String), ('FAMI_EDUCACIONMADRE', String), ('RENDIMIENTO_GLOBAL', String), ('coef_1', Float64), ('coef_2', Float64), ('coef_3', Float64), ('coef_4', Float64), ('prog_norm', String)])


ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_EDUCACIONMADRE,prog_norm
str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""enfermeria""","""bogota""","""entre 5.5 millones y menos de …","""menos de 10 horas""","""estrato 3""","""si""","""tecnica o tecnologica incomple…","""si""","""si""","""n""","""no""","""si""","""postgrado""","""enfermeria"""
"""derecho""","""atlantico""","""entre 2.5 millones y menos de …","""0""","""estrato 3""","""no""","""tecnica o tecnologica completa""","""si""","""no""","""n""","""no""","""si""","""tecnica o tecnologica incomple…","""derecho"""
"""mercadeo y publicidad""","""bogota""","""entre 2.5 millones y menos de …","""mas de 30 horas""","""estrato 3""","""si""","""secundaria (bachillerato) comp…","""si""","""no""","""n""","""no""","""no""","""secundaria (bachillerato) comp…","""mercadeo y publicidad"""
"""administracion de empresas""","""santander""","""entre 4 millones y menos de 5.…","""0""","""estrato 4""","""si""","""no sabe""","""si""","""no""","""n""","""no""","""si""","""secundaria (bachillerato) comp…","""administracion de empresas"""
"""psicologia""","""antioquia""","""entre 2.5 millones y menos de …","""entre 21 y 30 horas""","""estrato 3""","""si""","""primaria completa""","""si""","""si""","""n""","""no""","""si""","""primaria completa""","""psicologia"""
"""medicina veterinaria""","""antioquia""","""mas de 7 millones""","""menos de 10 horas""","""estrato 5""","""si""","""educacion profesional completa""","""si""","""si""","""n""","""no""","""si""","""secundaria (bachillerato) comp…","""medicina veterinaria"""
"""ingenieria mecanica""","""huila""","""entre 2.5 millones y menos de …","""entre 21 y 30 horas""","""estrato 2""","""si""","""educacion profesional incomple…","""si""","""si""","""n""","""si""","""si""","""tecnica o tecnologica completa""","""ingenieria mecanica"""
"""administracion en salud ocupac…","""bogota""","""entre 1 millon y menos de 2.5 …","""entre 11 y 20 horas""","""estrato 2""","""si""","""primaria incompleta""","""si""","""no""","""n""","""si""","""no""","""secundaria (bachillerato) inco…","""administracion en salud ocupac…"
"""ingenieria industrial""","""atlantico""","""entre 5.5 millones y menos de …","""menos de 10 horas""","""estrato 1""","""si""","""secundaria (bachillerato) comp…","""si""","""si""","""n""","""si""","""si""","""educacion profesional incomple…","""ingenieria industrial"""


## Codificaciones

In [22]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

In [23]:
rend_order   = ["bajo", "medio-bajo", "medio-alto", "alto"]
estrato_order= [f"estrato {i}" for i in range(1,6)]
horas_order  = ["0", "menos de 10 horas", "entre 11 y 20 horas",
                "entre 21 y 30 horas", "mas de 30 horas"]



Queremos categorizar según el tipo de columnas

In [24]:
rend_mapping   = {v: i for i, v in enumerate(rend_order)}
estrato_mapping = {v: i for i, v in enumerate(estrato_order)}
horas_mapping   = {v: i for i, v in enumerate(horas_order)}
binary_mapping  = {"no": 0, "si": 1}

In [27]:
ordinal_cols = {
    "RENDIMIENTO_GLOBAL": rend_order,
    "FAMI_ESTRATOVIVIENDA": estrato_order,
    "ESTU_HORASSEMANATRABAJA": horas_order,
}

binary_cols = [
    "FAMI_TIENEINTERNET",
    "FAMI_TIENELAVADORA",
    "FAMI_TIENEAUTOMOVIL",
    "ESTU_PRIVADO_LIBERTAD",
    "ESTU_PAGOMATRICULAPROPIO",
    "FAMI_TIENECOMPUTADOR",
]

nominal_cols = [
    "FAMI_EDUCACIONPADRE",
    "FAMI_EDUCACIONMADRE",
    "ESTU_PRGM_ACADEMICO",
    "ESTU_PRGM_DEPARTAMENTO",
    "prog_norm"
]

In [25]:
# Rendimiento global
rend_lookup = (
    pl.DataFrame({"RENDIMIENTO_GLOBAL": rend_order})
      .with_row_index("rendimiento_global_ord")  # 0, 1, 2, 3
      .with_columns(pl.col("rendimiento_global_ord").cast(pl.Int32))
      .lazy()
)

# Estrato de vivienda
estrato_lookup = (
    pl.DataFrame({"FAMI_ESTRATOVIVIENDA": estrato_order})
      .with_row_index("fami_estratovivienda_ord")
      .with_columns(pl.col("fami_estratovivienda_ord").cast(pl.Int32))
      .lazy()
)

# Horas semanales de trabajo
horas_lookup = (
    pl.DataFrame({"ESTU_HORASSEMANATRABAJA": horas_order})
      .with_row_index("estu_horassemanatrabaja_ord")
      .with_columns(pl.col("estu_horassemanatrabaja_ord").cast(pl.Int32))
      .lazy()
)

# Binarias
binary_lookup = (
    pl.DataFrame({"value": ["no", "si"], "bin": [0, 1]})
      .lazy()
)

In [28]:
df_mapped = (
    normalized_df
      .join(rend_lookup,    on="RENDIMIENTO_GLOBAL",     how="left")
      .join(estrato_lookup, on="FAMI_ESTRATOVIVIENDA",   how="left")
      .join(horas_lookup,   on="ESTU_HORASSEMANATRABAJA", how="left")
)

for col in binary_cols:
    df_mapped = (
        df_mapped
          .rename({col: "value"})
          .join(binary_lookup, on="value", how="left")
          .rename({"bin": f"{col.lower()}_bin", "value": col})
    )


In [32]:
df_sample = (
    df_mapped
      .collect()   
      .head(10)         
)

df_dummies = df_sample.to_dummies(
    columns=nominal_cols
)

columns_to_show = [
    "rendimiento_global_ord",
    "fami_estratovivienda_ord",
    "estu_horassemanatrabaja_ord",
    *[f"{c.lower()}_bin" for c in binary_cols],
    *[col for col in df_dummies.columns if col.startswith(tuple(nominal_cols))]
]

display(
    df_dummies.select(columns_to_show)
)

rendimiento_global_ord,fami_estratovivienda_ord,estu_horassemanatrabaja_ord,fami_tieneinternet_bin,fami_tienelavadora_bin,fami_tieneautomovil_bin,estu_privado_libertad_bin,estu_pagomatriculapropio_bin,fami_tienecomputador_bin,ESTU_PRGM_ACADEMICO_administracion de empresas,ESTU_PRGM_ACADEMICO_administracion en salud ocupacional,ESTU_PRGM_ACADEMICO_derecho,ESTU_PRGM_ACADEMICO_enfermeria,ESTU_PRGM_ACADEMICO_ingenieria industrial,ESTU_PRGM_ACADEMICO_ingenieria mecanica,ESTU_PRGM_ACADEMICO_medicina veterinaria,ESTU_PRGM_ACADEMICO_mercadeo y publicidad,ESTU_PRGM_ACADEMICO_psicologia,ESTU_PRGM_DEPARTAMENTO_antioquia,ESTU_PRGM_DEPARTAMENTO_atlantico,ESTU_PRGM_DEPARTAMENTO_bogota,ESTU_PRGM_DEPARTAMENTO_huila,ESTU_PRGM_DEPARTAMENTO_santander,FAMI_EDUCACIONPADRE_educacion profesional completa,FAMI_EDUCACIONPADRE_educacion profesional incompleta,FAMI_EDUCACIONPADRE_no sabe,FAMI_EDUCACIONPADRE_postgrado,FAMI_EDUCACIONPADRE_primaria completa,FAMI_EDUCACIONPADRE_primaria incompleta,FAMI_EDUCACIONPADRE_secundaria (bachillerato) completa,FAMI_EDUCACIONPADRE_tecnica o tecnologica completa,FAMI_EDUCACIONPADRE_tecnica o tecnologica incompleta,FAMI_EDUCACIONMADRE_educacion profesional incompleta,FAMI_EDUCACIONMADRE_postgrado,FAMI_EDUCACIONMADRE_primaria completa,FAMI_EDUCACIONMADRE_secundaria (bachillerato) completa,FAMI_EDUCACIONMADRE_secundaria (bachillerato) incompleta,FAMI_EDUCACIONMADRE_tecnica o tecnologica completa,FAMI_EDUCACIONMADRE_tecnica o tecnologica incompleta,prog_norm_administracion de empresas,prog_norm_administracion en salud ocupacional,prog_norm_derecho,prog_norm_enfermeria,prog_norm_ingenieria industrial,prog_norm_ingenieria mecanica,prog_norm_medicina veterinaria,prog_norm_mercadeo y publicidad,prog_norm_psicologia
i32,i32,i32,i64,i64,i64,i64,i64,i64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8
2,2,1,1,1,1,null,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
0,2,0,0,1,0,null,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0
0,2,4,1,1,0,null,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
3,3,0,1,1,0,null,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
1,2,3,1,1,1,null,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
2,4,1,1,1,1,null,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0
3,1,3,1,1,1,null,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
1,1,2,1,1,0,null,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
1,0,1,1,1,1,null,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [33]:
# Vamos a borrar la columna de estuvo privado de libertad. ya que todas son N en el dataset
df_dummies = df_dummies.drop("estu_privado_libertad_bin")

In [36]:
df_final = (
    df_mapped 
        .drop(nominal_cols)  # eliminamos las columnas originales
        .rename({
            "RENDIMIENTO_GLOBAL": "rendimiento_global",
            "FAMI_ESTRATOVIVIENDA": "fami_estratovivienda",
            "ESTU_HORASSEMANATRABAJA": "estu_horassemanatrabaja"
        })
        .collect()  # ejecutamos el lazy
        .to_dummies(columns=nominal_cols)  # one-hot encoding
)

In [40]:
import os

In [41]:
os.makedirs("preprocessed_data", exist_ok=True)
df_final.write_parquet("preprocessed_data/train_processed.parquet", compression="snappy")
print("✔️ Guardado en preprocessed_data/train_processed.parquet")

✔️ Guardado en preprocessed_data/train_processed.parquet
